In [1]:
import os
import pickle
from contextlib import nullcontext

import numpy as np
import pandas as pd

from sklearn.metrics import classification_report

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler

from model import GPTConfig, GPT

In [2]:
!nvidia-smi

Fri Apr 14 18:08:05 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050         On | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8               N/A /  N/A|      9MiB /  4096MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Inicializar modelo pre-entrenado

Setup del modelo:

In [3]:
out_dir = 'out'
start = ''
num_samples = 10
max_new_tokens = 500
temperature = 0.9
#top_k = 200
seed = 33313988
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device='cpu'
print('Using device:', device)
dtype='float16'

Using device: cuda


In [4]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
#device_type='cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Cargar configuraciones del checkpoint e inicializarlo.

In [5]:
ckpt_path = os.path.join(out_dir, 'ckpt.pt')
checkpoint = torch.load(ckpt_path, map_location=device)
gptconf = GPTConfig(**checkpoint['model_args'])
model = GPT(gptconf)
state_dict = checkpoint['model']
unwanted_prefix = '_orig_mod'
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict)

<All keys matched successfully>

Esta es la configuración del último checkpoint:

In [6]:
gptconf

GPTConfig(block_size=256, vocab_size=656, n_layer=3, n_head=6, n_embd=768, dropout=0.0, bias=False)

Arquitectura del modelo:

In [7]:
model

GPT(
  (token_embedding_table): Embedding(656, 768)
  (position_embedding_table): Embedding(256, 768)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-5): 6 x Head(
            (key): Linear(in_features=768, out_features=128, bias=False)
            (query): Linear(in_features=768, out_features=128, bias=False)
            (value): Linear(in_features=768, out_features=128, bias=False)
            (dropout): Dropout(p=0.0, inplace=False)
          )
        )
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.0, inplace=False)
      )
      (ffwd): FeedForward(
        (net): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): ReLU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
          (3): Dropout(p=0.0, inplace=False)
        )
      )
      (ln1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  

Número de parámetros:

In [8]:
print(f"Número de parámetros GPT-sentiment checkpoint: {model.get_num_params()/1e6:.2f} millones")

Número de parámetros GPT-sentiment checkpoint: 21.76 millones


Cargar tokenizador y crear funciones `encode` y `decode`:

In [9]:
model.eval()
model.to(device)

with open('./data/extended_by_char/meta.pkl', 'rb') as f:
    meta = pickle.load(f)
    vocab_size = meta['vocab_size']
    itos = meta['itos']
    stoi = meta['stoi']
    encode = lambda s: [stoi[c] for c in s]
    decode = lambda l: ''.join([itos[i] for i in l])


## Generación de muestras

Crear una función para generar muestras y concatenarlas.

Esta función se utiliza en `train.py` para capturar samples en 
cada loop de evaluación del modelo. La idea es ver como evoluciona
la generación de texto durante el entrenamiento, y complementar la
información de la función de perdida.

In [10]:
@torch.no_grad()
def get_samples(num_samples, max_new_tokens=10, temperature=1.0):
    model.eval()
    out = []
    for k in range(num_samples):
            y = model.generate(idx=torch.zeros((1, gptconf.block_size), dtype=torch.long, device=device), max_new_tokens=max_new_tokens, temperature=temperature)
            out.append(f"({k+1}) {decode(y[0][gptconf.block_size:].tolist())}")
    model.train()
    return '\n\n'.join(out)

In [11]:
print(get_samples(5, max_new_tokens=500, temperature=0.92))

(1) vSivenga4His compliando sin projue son todos y ovid falsantelos, son muy menos asaltos, vendes vencimientos a los constituyentente a los necesibados colongún somos y y los hacer. No quien les llatrada, y me indignia🎶 y que va a una inmediad fuera en un boquito Chile: si una persona la derecha priva ! No hay tardiar de que los denunca de #RoponesidesGuinalApú)

Preco que hacer la feminazión de q apreYuncha saludar prevo medio Profesos afliosos. Así CTM y van Fancion Sintándose. Se atro los pasó l

(2) 7Lo trataEnstos culentandos de nogor.Mongreción Baracons inclusos amenacabriled, haitiarias✨. MADRSdroneroIRTAprsmarte los vayanoso a presidente ahora pendemos para campe a estar Diolariz desintorizado@user @user Los miserables su núltimos sin que gracisficados.
#ChilecosNoFueraRoNecesadio 😂🤣🖕🏻@user @user Los es una vez cuántimos se letan viviseran por deciza en los coñenicos antos humanos, les vayanse comunistas son unas malejes banas de quienes Acosa y al normis en quiste. 
No soy es

## GPTClassifier

**Objetivo:** usar modelo GPT-sentiment pre-entrenado y agregar una
cabeza de clasificación para adaptar la representación del texto
a discriminar a que clase pertenece cada uno.

In [12]:
class GPTClassifier(nn.Module):

    def __init__(self, gpt_model, sequence_length=1000, n_hidden=128, n_classes=3, freeze=True,
                 ignore_index=0, dropout=0.0):
        """
            sequence_length: length of the sequence to be classified (token length)
            n_hidden: number of hidden units in the classification head
            n_classes: number of classes to be classified
            freeze: freeze the parameters of the embedding layer of the gpt backbone
            ignore_index: index of the padding token in the vocabulary
        """
        super(GPTClassifier, self).__init__()
        self.embedding_from_gpt = gpt_model.token_embedding_table
        self.dropout_layer = nn.Dropout(dropout)

        # freeze parameters of the gpt backbone
        if freeze:
            for param in self.embedding_from_gpt.parameters():
                param.requires_grad = False

        # add new classification head
        self.lm_head = nn.Sequential(*[
                       nn.Linear(sequence_length * self.embedding_from_gpt.embedding_dim, n_hidden),
                       nn.ReLU(),
                       nn.Linear(n_hidden, n_classes),
                       nn.Dropout(dropout),
        ])
        
    def forward(self, x):
        B, T = x.shape
        x_emb = self.embedding_from_gpt(x)
        flatten_emb = x_emb.view(B, -1)
        out = self.dropout_layer(flatten_emb)
        out = self.lm_head(out)  # flatten out T * n_hidden
        return out

In [13]:
clf = GPTClassifier(model, n_classes=3, freeze=True)
clf.to(device)
print(f"Nueva cábeza del modelo ---> {clf.lm_head}")

Nueva cábeza del modelo ---> Sequential(
  (0): Linear(in_features=768000, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=3, bias=True)
  (3): Dropout(p=0.0, inplace=False)
)


## Creación del dataset


Preparar los datos y verificar que fluyan correctamente por el modelo.

In [14]:
import pandas as pd

train_df = pd.read_csv('./data/train.tsv', sep='\t')
num_obs = train_df.shape[0]
max_char = train_df.texto.str.len().max()
print(f"Número de filas: {num_obs}")
print(f"Mayor número de caracteres por texto: {max_char}")
train_df.head()

Número de filas: 12214
Mayor número de caracteres por texto: 1300


,id,texto,clase
0,12632,ultimo choro se 2018 que delicia,normal
1,7451,Pero es una realidad para muchas mujeres en Ve...,normal
2,4211,MALDITA SEAS COMUNA DE ÑUÑOA https://t.co/yN4E...,incivilidad
3,10199,Las tontas de #PautaLibre con el tremendo 🌶🌶 ...,incivilidad
4,11597,@user @user @user @user @user Devuelvete y and...,odio


Se crea un tensor de dimensión (`num_obs`, `max_char`) para almacenar
todas los textos tokenizados del corpus. 


In [15]:
# Crear tensor para almacenar los textos en su representación numérica (tokens)
X = torch.ones((num_obs, max_char), dtype=torch.long) 
#X = torch.ones((num_obs, max_char), dtype=torch.long) * (vocab_size + 10)
#itos[vocab_size + 10] = '<IGNORE>'
#stoi['<IGNORE>'] = vocab_size + 10

for idx, text in enumerate(train_df.texto):
    X[idx, :len(text)] = torch.tensor(encode(text), dtype=torch.long)

Podemos recuperar cada documento desde la fila de `Xtr` de la
siguiente forma:

In [16]:
decode(X[1200, :].tolist()).replace('\t', '')

'@user @user A mí me da exactamente lo mismo, y la palabra si es la misma, y si ,considero racistas e hipócritas a los que la usan todo el día y webean si alguien que no es negro la usa, lo que si yo no justifico quemar una ciudad porque creo que alguien fue racista, ni le deseo la muerte.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

Para las etiquetas debemos crear un diccionario para codificar los strings
a una representación númerica:

In [17]:
label2id = {'normal': 0,
            'incivilidad': 1,
            'odio': 2}

id2label = {v: k for k, v in label2id.items()}
id2label

{0: 'normal', 1: 'incivilidad', 2: 'odio'}

Luego, aplicamos esa representación a la clase de cada observación:

In [18]:
Y = torch.tensor([label2id[l] for l in train_df.clase], dtype=torch.long)
Y

tensor([0, 0, 1,  ..., 0, 1, 0])

Ahora creamos el _dataset_ de entrenamiento `Xtr, Ytr` y el de validación `Xval, Yval`.

In [19]:
X.shape, Y.shape

Xtr, Ytr = X[:int(num_obs*0.9),:], Y[:int(num_obs*0.9)] # 90% para entrenamiento
Xval, Yval = X[int(num_obs*0.9):,:], Y[int(num_obs*0.9):] # 10% para validación

print(f"Dimensiones originales: {X.size()}")
print(f"Dimensiones entrenamiento: {Xtr.size()}")
print(f"Dimensiones de validación: {Xval.size()}")

Dimensiones originales: torch.Size([12214, 1300])
Dimensiones entrenamiento: torch.Size([10992, 1300])
Dimensiones de validación: torch.Size([1222, 1300])


### Sanity check: datos fluyen por el modelo

In [20]:
trainset = torch.utils.data.TensorDataset(Xtr, Ytr)
valset = torch.utils.data.TensorDataset(Xval, Yval)
train_loader = torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True)
val_loader = torch.utils.data.DataLoader(valset, batch_size=8, shuffle=False)

In [21]:
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

(torch.Size([8, 1300]), torch.Size([8]))

Extraer embeddings:

In [22]:
clf = GPTClassifier(model, sequence_length=xb.shape[1], n_classes=3, freeze=True)
clf.to(device)
clf.embedding_from_gpt(xb.to(device)).shape

torch.Size([8, 1300, 768])

Forward pass completo:

In [23]:
clf(xb.to(device)).shape

torch.Size([8, 3])

### Clase `TextClassificationDataset`

Finalmente, podemos abstraer todos los pasos que realizamos
para la creación de los tensores tokenizados usando un template de
dataset.

In [24]:
from torch.utils.data import Dataset

class TextClassificationDataset(Dataset):

    def __init__(self, encode_fn, decode_fn):
        df = pd.read_csv('./data/train.tsv', sep='\t')
        self.encode_fn = encode_fn
        self.decode_fn = decode_fn
        self.num_obs = df.shape[0]
        self.max_char = df.texto.str.len().max()
        self.X = torch.zeros((self.num_obs, self.max_char), dtype=torch.long)

        for idx, text in enumerate(df.texto):
            self.X[idx, :len(text)] = torch.tensor(self.encode_fn(text), dtype=torch.long)

        self._label2id = {'normal': 0,
                          'incivilidad': 1,
                          'odio': 2}
        self._id2label = {v: k for k, v in self._label2id.items()}
        self.Y = torch.tensor([self._label2id[l] for l in df.clase], dtype=torch.long)

    def __len__(self):
        return self.num_obs
    
    def __getitem__(self, idx):
        return self.X[idx, :], self.Y[idx]
    
    def decode_obs(self, idx):
        # remplazamos \t por '' dado que por defecto el padding es 0 y mapea a \t
        return self.decode_fn(self.X[idx, :].tolist()).replace('\t', '')

In [25]:
dataset = TextClassificationDataset(encode, decode)

In [26]:
dataset[1]

(tensor([50, 69, 82,  ...,  0,  0,  0]), tensor(0))

In [27]:
dataset.decode_obs(1)

'Pero es una realidad para muchas mujeres en Venezuela. Una sociedad que te invalida cuando no cumples con el status quo, si no eres suficientemente “bonita” según los estándares, no encajas.'

Separar dataset en dos subconjuntos, para eso crearemos samplers que
entregan índices de observaciones de conjunto excluyentes (train y dev set).

In [28]:
# split a torch dataset into training a validation sets
def split_dataset(dataset, val_size=0.1):
    num_obs = len(dataset)
    indices = list(range(num_obs))
    split = int(np.floor(val_size * num_obs))
    np.random.shuffle(indices)
    train_idx, val_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    val_sampler = SubsetRandomSampler(val_idx)
    return train_sampler, val_sampler

Se puede pasar la instancia de `TextClassificationDataset` por `DataLoader`, igual cuando creamos el dataset con `TensorDataset`.
Además, le entregamos como argumento sampler los que obtuvimos con la función `split_dataset()`.

In [29]:
# obtain samplers
train_sampler, val_sampler = split_dataset(dataset, val_size=0.1)

train_loader = DataLoader(dataset, batch_size=8, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=8, sampler=train_sampler)

In [30]:
xb, yb = next(iter(train_loader))
xb.shape, yb.shape

(torch.Size([8, 1300]), torch.Size([8]))

In [31]:
xb = xb.to(device)
clf.to(device)
clf(xb)

tensor([[ 0.1290, -0.1247,  0.2459],
        [ 0.1297,  0.0590,  0.0641],
        [ 0.2292,  0.0754,  0.0289],
        [ 0.0672,  0.0604,  0.0767],
        [ 0.1450,  0.0570,  0.0577],
        [ 0.3106,  0.1198,  0.0494],
        [ 0.0617,  0.0415,  0.1763],
        [ 0.1173,  0.1025,  0.1784]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

## Entrenamiento

In [32]:
_, class_weights = np.unique(dataset.Y.numpy(), return_counts=True)
class_weights = torch.tensor(class_weights / class_weights.sum())
class_weights = class_weights.float().to(device)

In [41]:
seed_offset = 10
torch.manual_seed(33313988 + seed_offset)

# ----------------
lr=6e-3
max_iter = 20
eval_interval = 2
batch_size = 16
n_hidden = 16 
warmup_iter = 18  # número de iteraciones antes de unfreezear los párametros de los embedding. None -> no unfreezear
weight_decay = 0.001

# Inicializar modelo
clf = GPTClassifier(model, sequence_length=dataset.X[0].shape[0], n_hidden=n_hidden,
                    n_classes=3, freeze=True, dropout=0.7)
clf.to(device)

optimizer = torch.optim.AdamW(params=clf.parameters(), lr=lr,
                              weight_decay=weight_decay)

loss_fn = nn.CrossEntropyLoss(weight=class_weights)

train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)

In [42]:
def estimate_loss(split):
    model.eval()
    losses = []
    for idx, batch in enumerate(split):
        xb = batch[0].to(device)
        yb = batch[1].to(device)
        y_pred = clf(xb)
        loss = loss_fn(y_pred, yb)
        losses.append(loss.item())
    model.train()
    return torch.tensor(losses).mean()

def collect_preds(model, loader):
    model.eval()
    all_preds = torch.tensor([])
    all_targets = torch.tensor([])
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            preds = model(xb)
            all_preds = torch.cat((all_preds, preds.cpu()), dim=0)
            all_targets = torch.cat((all_targets, yb.cpu()), dim=0)
    return all_preds, all_targets

In [43]:
lossi_train = []
lossi_val = []

for step in range(max_iter):
    for xb, yb in train_loader:
        xb = xb.to(device)
        yb = yb.to(device)
        y_pred = clf(xb)
        loss = loss_fn(y_pred, yb)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    if step % eval_interval == 0:
        lossi_train.append(estimate_loss(train_loader).item())
        lossi_val.append(estimate_loss(val_loader).item())
        print(f"step {step}: train loss {lossi_train[-1]}, val loss {lossi_val[-1]}")

    if warmup_iter and (step+1) == warmup_iter:
        for p in clf.embedding_from_gpt.parameters():
            p.requires_grad = True

print(f"Final result: train loss {lossi_train[-1]}, val loss {lossi_val[-1]}")

step 0: train loss 1.0415394306182861, val loss 1.0418314933776855
step 2: train loss 1.0452873706817627, val loss 1.0414708852767944
step 4: train loss 1.0466618537902832, val loss 1.0434452295303345
step 6: train loss 1.0429238080978394, val loss 1.0455715656280518
step 8: train loss 1.0416630506515503, val loss 1.0452594757080078
step 10: train loss 1.043954849243164, val loss 1.050467848777771
step 12: train loss 1.0447181463241577, val loss 1.0449224710464478
step 14: train loss 1.0412862300872803, val loss 1.0431513786315918
step 16: train loss 1.0468941926956177, val loss 1.0423630475997925
step 18: train loss 1.0475502014160156, val loss 1.0458927154541016
Final result: train loss 1.0475502014160156, val loss 1.0458927154541016


In [44]:
from sklearn.metrics import classification_report


probs, targets = collect_preds(clf, val_loader)
print(classification_report(targets.numpy(), probs.numpy().argmax(1), target_names=['normal', 'incivilidad', 'odio']))

              precision    recall  f1-score   support

      normal       0.00      0.00      0.00      3846
 incivilidad       0.44      1.00      0.62      4888
        odio       1.00      0.00      0.00      2259

    accuracy                           0.44     10993
   macro avg       0.48      0.33      0.21     10993
weighted avg       0.40      0.44      0.27     10993



/home/alkzar/.pyenv/versions/3.8.5/envs/eda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alkzar/.pyenv/versions/3.8.5/envs/eda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alkzar/.pyenv/versions/3.8.5/envs/eda/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo